# Creating an Availability Group (AG) Using DBATools
## Prerequisites
* At least two servers joined to a Windows Server Failover Cluster without shared storage
* Each server must be running Enterprise or Developer Edition and must be the same version
* The log and data paths should be the same
* Enable Always On Availability Groups for each instance to be added - NOTE: You can enable Always On Availability Groups from SQL Server Configuration Manager or by using the DBATools cmdlet ```
Enable-DbaAgHadr.
```
* Each database to be added to the Availability Group must be in full recovery and have a full backup run

This example uses Automatic Seeding, which is the easiest way to set up an AG.  You can find information about the other seeding options [here](https://docs.microsoft.com/en-us/sql/database-engine/availability-groups/windows/use-the-availability-group-wizard-sql-server-management-studio?view=sql-server-ver15).

## Building an AG Using DBATools

Once the prerequisites are met, ```New-DbaAvailabilityGroup``` builds your AG with a single command.




In [1]:
$params = @{
 Primary = 'fbgtrisqlagvm2'
 Secondary = 'fbgtrisqlagvm3'
 Name = 'fbgtrisqlag1'
 Database= 'WideWorldImporters'
 SeedingMode = 'Automatic'
 DatabaseHealthTrigger = $true
 DtcSupport = $true
 Confirm = $false
 }

New-DbaAvailabilityGroup @params;



ComputerName               : fbgtrisqlagvm2
InstanceName               : MSSQLSERVER
SqlInstance                : fbgtrisqlagvm2
LocalReplicaRole           : Primary
AvailabilityGroup          : fbgtrisqlag1
PrimaryReplica             : fbgtrisqlagvm2
ClusterType                : Wsfc
DtcSupportEnabled          : True
AutomatedBackupPreference  : Secondary
AvailabilityReplicas       : {fbgtrisqlagvm2, fbgtrisqlagvm3}
AvailabilityDatabases      : {WideWorldImporters}
AvailabilityGroupListeners : {}





## Create Dynamic Network Name Listener
In 2019, Microsoft introduced the Dynamic Network Name listener.  This removes the need for a load balancer when creating an Availability Group listener for an AG in Azure.  The following script will create a DNN listener.

In [3]:
$ag = 'fbgtrisqlag1';
$dns = 'fbgtrisqlaglist';
$port = '1437';

$ErrorActionPreference = "Stop"

# create the DNN resource with the port as the resource name
Add-ClusterResource -Name $Port -ResourceType "Distributed Network Name" -Group $Ag 

# set the DNS name of the DNN resource
Get-ClusterResource -Name $Port | Set-ClusterParameter -Name DnsName -Value $Dns 

# start the DNN resource
Start-ClusterResource -Name $Port

$Dep = Get-ClusterResourceDependency -Resource $Ag
if ( $Dep.DependencyExpression -match '\s*\((.*)\)\s*' )
{
    $DepStr = "$($Matches.1) or [$Port]";
}
else
{
    $DepStr = "[$Port]";
}

# add the Dependency from availability group resource to the DNN resource
Set-ClusterResourceDependency -Resource $Ag -Dependency "$DepStr";


#bounce the AG resource
Stop-ClusterResource -Name $Ag;
Start-ClusterResource -Name $Ag;

Name State   OwnerGroup   ResourceType            
---- -----   ----------   ------------            
1437 Offline fbgtrisqlag1 Distributed Network Name
1437 Online  fbgtrisqlag1 Distributed Network Name


f... Offline fbgtrisqlag1 SQL Server Availabili...


f... Online  fbgtrisqlag1 SQL Server Availabili...




## Create Agent Jobs to Test Sync-DbaAvailabilityGroup

One of the challenges of running an AG is keeping non-database objects like logins and jobs synced between the replicas.  DbaTools provides cmdlet ```Sync-DbaAvailabilityGroup``` to maintain replicas.  To test this, I will use ```Install-DbaMaintenanceSolution``` to install Ola Hallengren's Maintenance Solution jobs.

In [ ]:
Install-DbaMaintenanceSolution -SqlInstance fbgtrisqlagvm2 -Database master -ReplaceExisting -InstallJobs;

Confirm the jobs have been created using `Get-DbaAgentJob`. The jobs in the Maintenance Solution are all in category Database Maintenance.  Also, check the job list on the secondary replica.

In [ ]:
Get-DbaAgentJob -SqlInstance fbgtrisqlagvm2 -Category 'Database Maintenance' | Select ComputerName, Name;
Get-DbaAgentJob -SqlInstance fbgtrisqlagvm3 -Category 'Database Maintenance' | Select ComputerName, Name;

Once the jobs have been created, sync them to the secondary replica using ```Sync-DbaAvailabilityGroup```. The cmdlet will sync the following object types:
* SpConfigure 
* CustomErrors 
* Credentials 
* DatabaseMail 
* LinkedServers 
* Logins 
* LoginPermissions 
* SystemTriggers 
* DatabaseOwner 
* AgentCategory 
* AgentOperator 
* AgentAlert 
* AgentProxy 
* AgentSchedule 
* AgentJob

For this example, everything but Agent jobs will be excluded.   

In [ ]:
$primary = 'fbgtrisqlagvm2';
$secondary = 'fbgtrisqlagvm3';
$ag = 'fbgtrisqlag1';
$exclude = "SpConfigure","CustomErrors","Credentials","DatabaseMail","LinkedServers","Logins","LoginPermissions","SystemTriggers","DatabaseOwner","AgentCategory","AgentOperator","AgentAlert","AgentProxy","AgentSchedule";

Sync-DbaAvailabilityGroup -Primary $primary -Secondary $secondary -AvailabilityGroup $ag -exclude $exclude;

## Failover Availability Group

Once the secondary replica has been synced, the AG is ready to be failed over. The ```Invoke-DbaAgFailover``` cmdlet will perform the failover.

In [ ]:
$newprimary = 'fbgtrisqlagvm3';
$oldprimary = 'fbgtrisqlagvm2'
$ag = 'fbgtrisqlag1';
Invoke-DbaAgFailover -SqlInstance $newprimary -AvailabilityGroup $ag -Force;
Resume-DbaAgDbDataMovement -SqlInstance $oldprimary -AvailabilityGroup $ag -Database WideWorldImporters -Confirm:$false;